In [1]:
import time
import os
import pandas as pd
import numpy as np
import sqlalchemy.schema
from sqlalchemy import create_engine, exc, types

### Common Variables

In [2]:
SS_CONNECTION_STRING = 'mssql+pyodbc://SUBDN748/BubeTests?trusted_connection=yes&driver=ODBC+Driver+17+for+SQL+Server'
# System folders
FolderPath = 'C:\\Users\\GUILARRA\\Documents\\SQL\\BubeTests\\'
PD1PFile = 'PD1P_SS.csv'
NettingNegative = 'netting Negative at Counterparty_Deal level.xlsx'
NettingPositive = 'netting Positive at Counterparty_Deal level.xlsx'

## Read CSV Files functions

In [3]:
def read_csv_file(filepath, skip_rows: 0):
    """read csv file
    :param str filepath: should contain file path + file name
    :param int skip_rows: number of rows to skip in the header
    :param str file_name: name of the file"""
    df = pd.read_csv(filepath, sep=",", skiprows=skip_rows)
    return df


In [4]:
df_pd1p = read_csv_file(FolderPath + PD1PFile, 0)
# Convert the column from datetime to date
df_pd1p['TradeDate'] = pd.to_datetime(df_pd1p['TradeDate']).dt.date

# Add a new column called "ReportingDate" with the current date
df_pd1p['ReportingDate'] = pd.to_datetime('today').date()
df_pd1p.sample(5)

,SubStrategy,BookingFinal,Unrealised Full Amount,CounterpartyPortfolio,CounterParty,CurveLevelMax,InstrumentNameSingle,TradeDate,ReportingDate
1193,NaN,Yes,755241.67,ST_PL_TRAD,EGL-PL,3,Gas OTC Purchase,2021-11-29,2023-07-06
45,Macquarie,Yes,2420048.07,MGMT_ORI_RW,EGLCHE,3,Power Financial Swap,2022-01-18,2023-07-06
594,NaN,Yes,13179174.88,NaN,MTSTONGE,3,Power Metered Physical,2021-05-18,2023-07-06
762,NaN,Yes,2235156.17,NaN,CEGUNOPO,3,pwr trans fs,2022-05-11,2023-07-06
85,NaN,Yes,7776.76,CT_CH_GRN,EGLCHE,3,Green Certificates,2022-05-16,2023-07-06


## Read Excel Files

In [4]:
def read_excel_file(filepath, sheet_name, skip_rows: 0, use_columns=None):
    """read excel file
    :param str filepath: should contain file path + file name
    :param str sheet_name: name of the Excel sheet to be analyzed
    :param int skip_rows: number of rows to skip in the header
    :param list[str] use_columns: array of columns to be extracted, none by default"""
    df = pd.read_excel(filepath, sheet_name=sheet_name, skiprows=skip_rows, usecols=use_columns)
    return df

In [6]:
df_NN = read_excel_file(FolderPath + NettingNegative,'Sheet1', 0)
df_NN.sample(5)

,ST_LT_ALLOCATION.ST_LT,Portfolio Type,SAP Partner Type,Counterparty,ENDUR_ID,Gross liabilities before balance sheet netting,Netting,"Net liabilities, reported on the balance sheet",Liabilities which are not subject to netting agreements or which are not subject to legally enforcable netting agreements,Total liabilities recognized on the balance sheet,Financial assets,Liabilities after recognition of the netting-potential
37281,ST,TR,Int.Bücher,EGLCHE,16705392,0.000000e+00,0.000000e+00,0.0,-3.735102e+06,-3.735102e+06,0.0,-3.735102e+06
51882,ST,TR,Third Party,GRUNWARM,17308535,-8.162185e+05,8.162185e+05,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00
24732,ST,TR,Int.Bücher,EGL-PL,17848709,0.000000e+00,0.000000e+00,0.0,-3.448484e+05,-3.448484e+05,0.0,-3.448484e+05
33016,ST,TR,Int.Bücher,EGLCHE,14759051,0.000000e+00,0.000000e+00,0.0,-1.170577e+05,-1.170577e+05,0.0,-1.170577e+05
42460,ST,TR,Third Party,ALPIQ-AG,13748178,-1.596145e+06,1.596145e+06,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00


In [7]:
df_NN = df_NN.rename(columns={
    'ST_LT_ALLOCATION.ST_LT': 'STLT_ID',
    'Portfolio Type': 'PortfolioType',
    'SAP Partner Type': 'SAPPartnerType',
    'Counterparty': 'Counterparty',
    'ENDUR_ID': 'EndurDealID',
    'Gross liabilities before balance sheet netting': 'Netting1',
    'Netting': 'Netting2',
    'Net liabilities, reported on the balance sheet': 'Netting3',
    'Liabilities which are not subject to netting agreements or which are not subject to legally enforcable netting agreements': 'Netting4',
    'Total liabilities recognized on the balance sheet': 'Netting5',
    'Financial assets': 'Netting6',
    'Liabilities after recognition of the netting-potential': 'Netting7'
})
df_NN['ReportingDate'] = pd.to_datetime('today').date()

In [8]:
df_NP = read_excel_file(FolderPath + NettingPositive,'Sheet1', 0)
df_NP.sample(5)

,ST_LT_ALLOCATION.ST_LT,Portfolio Type,SAP Partner Type,Counterparty,ENDUR_ID,Gross assets before balance sheet netting,Netting,"Net assets, reported on the balance sheet",Assets that are not subject to master netting agreements or are not subject to legally enforcable master netting agreements,Total assets recognized on the balance sheet,Financial liabilities,Assets after recognition of the netting-potential
4893,LT,TR,Third Party,EDF-TRAD,15223457,2.583035e+06,-2.583035e+06,0.0,0.000000e+00,0.000000e+00,0.0,0.000000e+00
48835,ST,TR,Third Party,IRGIT,12931727,0.000000e+00,0.000000e+00,0.0,1.011106e+06,1.011106e+06,0.0,1.011106e+06
29474,ST,TR,Int.Bücher,EGLCHE,14929766,0.000000e+00,0.000000e+00,0.0,2.424650e+06,2.424650e+06,0.0,2.424650e+06
14436,ST,TR,Int.Bücher,EGL-PL,10550196,0.000000e+00,0.000000e+00,0.0,9.941683e+05,9.941683e+05,0.0,9.941683e+05
33639,ST,TR,Int.Bücher,EGLCHE,16788494,0.000000e+00,0.000000e+00,0.0,6.593914e+05,6.593914e+05,0.0,6.593914e+05


In [9]:
df_NP =  df_NP.rename(columns={
    'ST_LT_ALLOCATION.ST_LT': 'STLT_ID',
    'Portfolio Type': 'PortfolioType',
    'SAP Partner Type': 'SAPPartnerType',
    'Counterparty': 'Counterparty',
    'ENDUR_ID': 'EndurDealID',
    'Gross assets before balance sheet netting': 'Netting1',
    'Netting': 'Netting2',
    'Net assets, reported on the balance sheet': 'Netting3',
    'Assets that are not subject to master netting agreements or are not subject to legally enforcable master netting agreements': 'Netting4',
    'Total assets recognized on the balance sheet': 'Netting5',
    'Financial liabilities': 'Netting6',
    'Assets after recognition of the netting-potential': 'Netting7'
})
df_NP['ReportingDate'] = pd.to_datetime('today').date()

## Insert into SQL Server functions

In [5]:
def connect_and_load_to_sqlserver(df, table_name: str, schema_name=None, if_exists='replace',column_types=None):
    """Function to connect and load a dataframe onto sqlserver sql:localhost
    :param df: dataframe to load to sql
    :param table_name: name of the table is going to be created
    :param schema_name: name of the schema in which the table will be allocated
    :param if_exists: in case you want to append change that parameter (replace/append)"""
    conn_string = SS_CONNECTION_STRING
    engine = create_engine(conn_string)
    start_time = time.time()
    # Create SQLAlchemy engine
    table_name_simple = os.path.splitext(table_name)[0]
    try:
        # Check if the schema exists
        #if not schema_exists(engine, schema_name):
        #    engine.execute(sqlalchemy.schema.CreateSchema(schema_name))
        #    raise ValueError(f"The schema '{schema_name}' does not exist.")
        df.to_sql(table_name_simple, schema=schema_name, con=engine, if_exists=if_exists, index=False,dtype=column_types)

    except exc.SQLAlchemyError as e:
        print('An error occurred while loading the data into the SQL Server table.')
        print(f'Error details: {str(e)}')

    return print("Table " + table_name + "." + schema_name + " inserted into sqlserver, duration: {} seconds".format(
        time.time() - start_time))

In [11]:
# Define the desired column types for D1P1
column_types = {
    'SubStrategy': types.String(length=255),
    'BookingFinal': types.String(length=255),
    'Unrealised Full Amount': types.Float(precision=2),
    'CounterpartyPortfolio': types.String(length=255),
    'CounterParty': types.String(length=255),
    'CurveLevelMax': types.Integer(),
    'InstrumentNameSingle': types.String(length=255),
    'TradeDate': types.Date(),
    'ReportingDate': types.Date()
}

In [12]:
# Define the desired column types for Netting Positive/Negative
column_types_netting = {
    'STLT_ID': types.String(length=255),
    'PortfolioType': types.String(length=255),
    'SAPPartnerType': types.String(length=255),
    'Counterparty': types.String(length=255),
    'EndurDealID': types.String(length=255),
    'Netting1': types.Float(precision=23),
    'Netting2': types.Float(precision=23),
    'Netting3': types.Float(precision=23),
    'Netting4': types.Float(precision=23),
    'Netting5': types.Float(precision=23),
    'Netting6': types.Float(precision=23),
    'Netting7': types.Float(precision=23),
    'ReportingDate': types.Date()
}

In [13]:
# Load CSV file
connect_and_load_to_sqlserver(df_pd1p, "PD1P_SS", "QV",'replace',column_types)

Table PD1P_SS.QV inserted into sqlserver, duration: 2.748042106628418 seconds


In [14]:
#Load Excel Files
connect_and_load_to_sqlserver(df_NN, "Netting_PositiveNegative", "QV",'append',column_types_netting)
connect_and_load_to_sqlserver(df_NP, "Netting_PositiveNegative", "QV",'append',column_types_netting)

Table Netting_PositiveNegative.QV inserted into sqlserver, duration: 106.63944435119629 seconds
Table Netting_PositiveNegative.QV inserted into sqlserver, duration: 97.04730296134949 seconds


### Twin Deals File

In [6]:
TwinDealsFile = "Twin Deals.xlsx"
df_TD = read_excel_file(FolderPath + TwinDealsFile,'Sheet1', 0)
df_TD.sample(5)

,TWIN_DEAL_ID,Twin original,PV Twin original,Twin generated,PV Twin generated,Total
8348,11448334_11448335,11448334,-9.396279e+03,11448335,9.396279e+03,0.0
33656,17602739_17602740,17602739,5.193313e+03,17602740,-5.193313e+03,0.0
2794,8861340_8861341,8861340,1.052468e+07,8861341,-1.052468e+07,0.0
14838,13594849_13594850,13594849,-9.775182e+04,13594850,9.775182e+04,0.0
23507,15574583_15574584,15574583,6.099397e+02,15574584,-6.099397e+02,0.0


In [7]:
# Define the desired column types for D1P1
column_types_TD = {
    'TWIN_DEAL_ID': types.String(length=255),
    'Twin original': types.String(length=255),
    'PV Twin original': types.Float(precision=23),
    'Twin generated': types.String(length=255),
    'PV Twin generated': types.Float(precision=23),
    'Total': types.Float(precision=23)
}

In [8]:
connect_and_load_to_sqlserver(df_TD, "TwinDeals", "QV",'replace',column_types_TD)

Table TwinDeals.QV inserted into sqlserver, duration: 68.7300021648407 seconds
